In [1]:
import os
import numpy as np
from keras.applications.resnet import ResNet50
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.optimizer_v1 import Adam

In [2]:
train_dir = r'D:\Dataset\data_mv\train'
test_dir = r'D:\Dataset\data_mv\test'

In [3]:
# Load and preprocess the training data
train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=4,
    class_mode='categorical',  # Assuming labels are provided as integers
    color_mode='rgb'  # Assuming images are grayscale
)

Found 48 images belonging to 3 classes.


In [4]:
# Load and preprocess the test data
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=4,
    class_mode='categorical',
    color_mode='rgb'
)

Found 24 images belonging to 3 classes.


In [5]:
x_train, y_train = train_generator.next()
x_test, y_test = test_generator.next()

In [7]:
# Load pre-trained VGG16 model (excluding top layers)
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(128, 128, 3))

94781440/94765736 [==============================] - 44s 0us/step


In [8]:
# Add custom top layers for classification
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
output = Dense(3, activation='softmax')(x)

In [9]:
# Create the new model
model = Model(inputs=base_model.input, outputs=output)

In [10]:
# Freeze base layers to prevent their weights from being updated
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']         

In [12]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Train the model
history=model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 13s 13s/step - loss: 1.4794 - accuracy: 0.0000e+00 - val_loss: 0.6289 - val_accuracy: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 60ms/step - loss: 0.6615 - accuracy: 1.0000 - val_loss: 0.2078 - val_accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 59ms/step - loss: 0.6135 - accuracy: 0.6667 - val_loss: 0.1790 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 59ms/step - loss: 0.5866 - accuracy: 0.6667 - val_loss: 0.2930 - val_accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 58ms/step - loss: 0.4838 - accuracy: 0.6667 - val_loss: 0.5265 - val_accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 55ms/step - loss: 0.4509 - accuracy: 1.0000 - val_loss: 0.6321 - val_accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 55ms/step - loss: 0.4374 - accuracy: 1.0000 - val_loss: 0.5031 - val_accuracy: 1.0000
Epoch 8/10
1/1 [=======

In [14]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print("Test accuracy:", test_acc)

1/1 [==============================] - 1s 536ms/step - loss: 5.1859 - accuracy: 0.2500
Test accuracy: 0.25


In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()